In [1]:
import argparse
import copy
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import roc_auc_score

from ml import learner, data
from ml.vision import utils
from pipe import constants, augmentations

In [2]:
test = pd.read_csv(constants.sample_submission_fpath)
train = pd.read_csv(constants.train_folds_fpath)

In [4]:
test_image_paths = [
    constants.data_path / f"test_256/{x}.jpg" for x in test.StudyInstanceUID.values
]
test_image_paths[:5]

[PosixPath('/home/gianluca/git/kaggle/ranzcr/data/test_256/1.2.826.0.1.3680043.8.498.46923145579096002617106567297135160932.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/test_256/1.2.826.0.1.3680043.8.498.84006870182611080091824109767561564887.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/test_256/1.2.826.0.1.3680043.8.498.12219033294413119947515494720687541672.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/test_256/1.2.826.0.1.3680043.8.498.84994474380235968109906845540706092671.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/test_256/1.2.826.0.1.3680043.8.498.35798987793805669662572108881745201372.jpg')]

In [5]:
train_image_paths = [
    constants.data_path / f"train_256/{x}.jpg" for x in train.StudyInstanceUID.values
]
train_image_paths[:5]

[PosixPath('/home/gianluca/git/kaggle/ranzcr/data/train_256/1.2.826.0.1.3680043.8.498.10980236531551060314989711662517682573.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/train_256/1.2.826.0.1.3680043.8.498.31644041654883936177483097186069054689.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/train_256/1.2.826.0.1.3680043.8.498.92698499133241834162015009420418179750.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/train_256/1.2.826.0.1.3680043.8.498.10069138128460654269681788885297696718.jpg'),
 PosixPath('/home/gianluca/git/kaggle/ranzcr/data/train_256/1.2.826.0.1.3680043.8.498.10994992736051720791598262985362954566.jpg')]

## Predict on training set

In [5]:
checkpoint_path = Path('../models/arch=resnest14d_sz=128_fold=0.ckpt')
pretrained_model = learner.ImageClassifier.load_from_checkpoint(
    checkpoint_path, in_channels=1, num_classes=11
)
pretrained_model.freeze()

In [6]:
list(pretrained_model.parameters())[0][0]

tensor([[[-0.3436, -0.5128, -0.4899],
         [-0.3492, -0.3446, -0.2687],
         [-0.3731, -0.2343, -0.2341]]])

In [11]:
list(pretrained_model.model.parameters())[0][0]

tensor([[[-0.3436, -0.5128, -0.4899],
         [-0.3492, -0.3446, -0.2687],
         [-0.3731, -0.2343, -0.2341]]], device='cuda:0')

In [7]:
hparams = argparse.Namespace(
    arch='resnest14d',
    sz=128,
    test_data=Path('../data/train_256'),
    aug='baseline'
)

train_aug, valid_aug, test_aug = augmentations.augmentations_factory(hparams)

dm = data.ImageDataModule(
    batch_size=256,
    test_image_paths=train_image_paths,
    test_augmentations=test_aug,
)
dm.setup()

outs = list(pretrained_model.predict_proba(dm.test_dataloader))
preds = np.vstack(outs)

train_preds = copy.copy(train)
train_preds[constants.target_cols] = preds

In [8]:
train_preds[constants.target_cols].mean()

ETT - Abnormal                0.003749
ETT - Borderline              0.046870
ETT - Normal                  0.230857
NGT - Abnormal                0.010472
NGT - Borderline              0.025475
NGT - Incompletely Imaged     0.074746
NGT - Normal                  0.177242
CVC - Abnormal                0.093819
CVC - Borderline              0.244747
CVC - Normal                  0.727748
Swan Ganz Catheter Present    0.027688
dtype: float64

In [9]:
train[constants.target_cols].mean()

ETT - Abnormal                0.002626
ETT - Borderline              0.037829
ETT - Normal                  0.240667
NGT - Abnormal                0.009274
NGT - Borderline              0.017585
NGT - Incompletely Imaged     0.091347
NGT - Normal                  0.159459
CVC - Abnormal                0.106206
CVC - Borderline              0.281222
CVC - Normal                  0.708839
Swan Ganz Catheter Present    0.027590
dtype: float64

In [10]:
from sklearn.metrics import roc_auc_score
roc_auc_score(
    y_true=train[constants.target_cols].values,
    y_score=train_preds[constants.target_cols].values,
    average='macro'
)

0.9136993636159523

## Predict on test set

In [10]:
hparams = argparse.Namespace(
    arch='resnest14d',
    sz=128,
    test_data=Path('../data/test_256'),
    aug='baseline'
)

train_aug, valid_aug, test_aug = augmentations.augmentations_factory(hparams)

dm = data.ImageDataModule(
    batch_size=256,
    test_image_paths=test_image_paths,
    test_augmentations=test_aug,
)
dm.setup()

outs = list(pretrained_model.predict_proba(dm.test_dataloader))
preds = np.vstack(outs)

test_preds = copy.copy(test)
test_preds[constants.target_cols] = preds
test_preds[constants.target_cols].mean()

ETT - Abnormal                0.001762
ETT - Borderline              0.019948
ETT - Normal                  0.104933
NGT - Abnormal                0.006625
NGT - Borderline              0.014579
NGT - Incompletely Imaged     0.033990
NGT - Normal                  0.085848
CVC - Abnormal                0.080980
CVC - Borderline              0.231591
CVC - Normal                  0.736716
Swan Ganz Catheter Present    0.011959
dtype: float64

## Train target distribution

In [11]:
train[constants.target_cols].mean()

ETT - Abnormal                0.002626
ETT - Borderline              0.037829
ETT - Normal                  0.240667
NGT - Abnormal                0.009274
NGT - Borderline              0.017585
NGT - Incompletely Imaged     0.091347
NGT - Normal                  0.159459
CVC - Abnormal                0.106206
CVC - Borderline              0.281222
CVC - Normal                  0.708839
Swan Ganz Catheter Present    0.027590
dtype: float64

## Replicate best scores for each fold

In [3]:
train_preds = copy.copy(train)
ARCH     = 'rexnet_200'
SZ       = 512
SZ_INPUT = SZ * 2
BS       = 16

for fold in range(5):
    checkpoint_path = Path(f'../models/arch={ARCH}_sz={SZ}_fold={fold}.ckpt')
    pretrained_model = learner.ImageClassifier.load_from_checkpoint(
        checkpoint_path, in_channels=1, num_classes=11
    )
    pretrained_model.freeze()
    
    hparams = argparse.Namespace(
        arch=ARCH,
        sz=SZ,
        test_data=Path(f'../data/train_{SZ_INPUT}'),
        aug='baseline'
    )

    train_aug, valid_aug, test_aug = augmentations.augmentations_factory(hparams)

    valid_image_paths = [
        constants.data_path / f"train_{SZ_INPUT}/{x}.jpg" for x in train[train.kfold==fold].StudyInstanceUID.values
    ]
    valid_image_paths[:5]
    
    dm = data.ImageDataModule(
        batch_size=BS,
        test_image_paths=valid_image_paths,
        test_augmentations=valid_aug,
    )
    dm.setup()

    outs = list(pretrained_model.predict_proba(dm.test_dataloader))
    preds = np.vstack(outs)

    break

In [4]:
preds.shape

(6015, 11)

In [5]:
from sklearn.metrics import roc_auc_score
roc_auc_score(
    y_true=train[train.kfold==fold][constants.target_cols].values,
    y_score=preds,
    average='macro'
)

0.8408216950745881

In [28]:
from sklearn.metrics import roc_auc_score
roc_auc_score(
    y_true=train[train.kfold==fold][constants.target_cols].values,
    y_score=preds,
    average='macro'
)

0.9491113828429192

In [11]:
pretrained_model.__dict__

{'training': False,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('model',
               ReXNetV1(
                 (stem): ConvBnAct(
                   (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
                   (bn): BatchNormAct2d(
                     64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
                     (act): SiLU(inplace=True)
                   )
                 )
                 (features): Sequential(
                   (0): LinearBottleneck(
                     (conv_dw): ConvBnAct(
                       (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
                       (bn): 